## Save parquet with pick info with event info included

In [1]:
import obspy
import matplotlib.pyplot as plt
import numpy as np
import glob
import pandas as pd
import time; import datetime
from obspy.core.utcdatetime import UTCDateTime
from obspy.clients.fdsn.client import Client
from collections import defaultdict
from obspy import read_inventory
import pyarrow
import obsplus

In [2]:
ml_files = glob.glob('catalog_css/**/XO_*.quakeml')
print(ml_files)

['catalog_css/2019_01/XO_2019_01.quakeml', 'catalog_css/2018_11/XO_2018_11.quakeml', 'catalog_css/2018_08/XO_2018_08.quakeml', 'catalog_css/2019_03/XO_2019_03.quakeml', 'catalog_css/2019_06/XO_2019_06.quakeml', 'catalog_css/2018_05/XO_2018_05.quakeml', 'catalog_css/2018_07/XO_2018_07.quakeml', 'catalog_css/2019_07/XO_2019_07.quakeml', 'catalog_css/2018_06/XO_2018_06.quakeml', 'catalog_css/2018_12/XO_2018_12.quakeml', 'catalog_css/2018_09/XO_2018_09.quakeml', 'catalog_css/2019_05/XO_2019_05.quakeml', 'catalog_css/2019_02/XO_2019_02.quakeml', 'catalog_css/2019_04/XO_2019_04.quakeml', 'catalog_css/2018_10/XO_2018_10.quakeml']


In [3]:
count = 0
cat = obspy.core.event.Catalog()
for file in ml_files:
    count += 1
    print(count)
    cat.extend(obspy.core.event.read_events(file))

1
2


/srv/conda/envs/alaska-ml/lib/python3.9/site-packages/obspy/io/quakeml/core.py:184: UserWarning: Could not convert 2018-11-14T00:42:60.000Z to type <class 'obspy.core.utcdatetime.UTCDateTime'>. Returning None.
  warnings.warn(msg % (text, convert_to))


3
4
5
6
7
8
9


/srv/conda/envs/alaska-ml/lib/python3.9/site-packages/obspy/io/quakeml/core.py:184: UserWarning: Could not convert 2018-06-18T21:56:60.000Z to type <class 'obspy.core.utcdatetime.UTCDateTime'>. Returning None.
  warnings.warn(msg % (text, convert_to))


10
11
12
13


/srv/conda/envs/alaska-ml/lib/python3.9/site-packages/obspy/io/quakeml/core.py:184: UserWarning: Could not convert 2021-02-09T02:50:60.000Z to type <class 'obspy.core.utcdatetime.UTCDateTime'>. Returning None.
  warnings.warn(msg % (text, convert_to))


14


/srv/conda/envs/alaska-ml/lib/python3.9/site-packages/obspy/io/quakeml/core.py:184: UserWarning: Could not convert 2021-05-11T20:56:60.000Z to type <class 'obspy.core.utcdatetime.UTCDateTime'>. Returning None.
  warnings.warn(msg % (text, convert_to))


15


/srv/conda/envs/alaska-ml/lib/python3.9/site-packages/obspy/io/quakeml/core.py:184: UserWarning: Could not convert 2020-11-19T01:12:60.000Z to type <class 'obspy.core.utcdatetime.UTCDateTime'>. Returning None.
  warnings.warn(msg % (text, convert_to))


In [125]:
from obspy import read_inventory
sta_metadata = read_inventory('alaska_stations.xml',format='STATIONXML')

In [133]:
station_locs = defaultdict(dict)
for network in sta_metadata:
    for station in network:
        for chn in station:
            sid = f"{network.code}.{station.code}.{chn.location_code}.{chn.code[:-1]}" + chn.start_date.strftime('%Y%j')
            if sid in station_locs:
                station_locs[sid]["component"] += f",{chn.code[-1]}"
                station_locs[sid]["response"] += f",{chn.response.instrument_sensitivity.value:.2f}"
            else:
                component = f"{chn.code[-1]}"
                response = f"{chn.response.instrument_sensitivity.value:.2f}"
                dtype = chn.response.instrument_sensitivity.input_units.lower()
                tmp_dict = {}
                tmp_dict["longitude"], tmp_dict["latitude"], tmp_dict["elevation(m)"] = (
                    chn.longitude,
                    chn.latitude,
                    chn.elevation,
                )
                tmp_dict["component"], tmp_dict["response"], tmp_dict["unit"] = component, response, dtype
                tmp_dict["start_date"], tmp_dict["end_date"] = chn.start_date,chn.end_date
                tmp_dict["id2"] = f"{network.code}.{station.code}.{chn.location_code}.{chn.code[:-1]}"
                if tmp_dict["end_date"] is None:
                    tmp_dict["end_date"] = UTCDateTime(2100,1,1)
                tmp_dict["network"], tmp_dict["station"] = network.code, station.code
                station_locs[sid] = tmp_dict

station_locs = pd.DataFrame.from_dict(station_locs,orient='index')
station_locs["id"] = station_locs.index

In [ ]:
from collections import defaultdict

# Set up empty dictionary
dd = defaultdict(list)
# Get station list
dfS = pd.read_parquet('https://github.com/zoekrauss/alaska_catalog/raw/main/data_acquisition/alaska_stations.parquet')

for e in cat.events:
    arrs = e.origins[0].arrivals
    print(str(e.origins[0].time))
    for count,p in enumerate(e.picks):
        
        # Pick/arrival info
        arr = [a for a in arrs if str(a.pick_id) == str(p.resource_id)]
        dd['phase'].append(arr[0].phase)
        dd['azimuth'].append(arr[0].azimuth)
        dd['distance'].append(arr[0].distance)
        dd['time_residual'].append(arr[0].time_residual)
        dd['time_weight'].append(arr[0].time_weight)
        dd['pick_time'].append(str(p.time))
        dd['network'].append(p.waveform_id.network_code)
        dd['station'].append(p.waveform_id.station_code)
        dd['channel'].append(p.waveform_id.channel_code)
        dd['pick_id'].append(str(p.resource_id))
        sta_code = f"{p.waveform_id.network_code}.{p.waveform_id.station_code}..{p.waveform_id.channel_code[:-1]}"
        dd['sta_code'].append(sta_code)
        location_code = p.waveform_id.location_code
        
        # Event info
        dd['ev_id'].append(str(e.resource_id))
        dd['ev_time'].append(str(e.origins[0].time))
        dd['ev_longitude'].append(e.origins[0].longitude)
        dd['ev_latitude'].append(e.origins[0].latitude)
        dd['ev_depth'].append(e.origins[0].depth)
        dd['ev_magnitude'].append(e.magnitudes[0].mag)
        dd['mag_type'].append(e.magnitudes[0].magnitude_type)
        dd['ev_phasecount'].append(e.origins[0].quality.used_phase_count)
        
        # Station info
        sta = station_locs[station_locs.id2 == sta_code]
        if sta.empty:
            dd['sta_lon'].append(0.0)
            dd['sta_lat'].append(0.0)
            dd['sta_elev'].append(0.0)
            dd['sta_unit'].append('')
            continue
        dd['sta_lon'].append(sta.iloc[0]['longitude'])
        dd['sta_lat'].append(sta.iloc[0]['latitude'])
        dd['sta_elev'].append(sta.iloc[0]['elevation(m)'])
        dd['sta_unit'].append(sta.iloc[0]['unit'])
        
        
        

In [155]:
alll = pd.DataFrame.from_dict(dd)
alll

,phase,azimuth,distance,time_residual,time_weight,pick_time,network,station,channel,pick_id,...,ev_longitude,ev_latitude,ev_depth,ev_magnitude,mag_type,ev_phasecount,sta_lon,sta_lat,sta_elev,sta_unit
0,P,6.76,0.411,-0.014,0.265,2019-01-01T05:06:05.601000Z,XO,LT20,HHZ,quakeml:earthquake.alaska.edu/pick/95,...,-161.4584,54.3925,25600.0,2.4,ml,35,-161.374600,54.800200,-150.0,m/s
1,P,76.00,0.751,0.343,0.265,2019-01-01T05:06:10.941000Z,XO,LT17,HHZ,quakeml:earthquake.alaska.edu/pick/97,...,-161.4584,54.3925,25600.0,2.4,ml,35,-160.201600,54.567700,-114.0,m/s
2,P,106.40,0.733,1.011,0.265,2019-01-01T05:06:11.047000Z,XO,WS75,HHZ,quakeml:earthquake.alaska.edu/pick/98,...,-161.4584,54.3925,25600.0,2.4,ml,35,-160.256908,54.179630,-1108.9,m/s
3,P,132.56,0.895,-1.155,0.265,2019-01-01T05:06:11.579000Z,XO,WD69,HHZ,quakeml:earthquake.alaska.edu/pick/99,...,-161.4584,54.3925,25600.0,2.4,ml,35,-160.342890,53.782065,-3895.9,m/s
4,S,6.76,0.411,0.220,0.106,2019-01-01T05:06:12.909000Z,XO,LT20,HH2,quakeml:earthquake.alaska.edu/pick/100,...,-161.4584,54.3925,25600.0,2.4,ml,35,-161.374600,54.800200,-150.0,m/s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403726,S,319.33,2.394,-2.320,0.128,2018-10-31T20:04:46.407000Z,XO,KD01,HHE,quakeml:earthquake.alaska.edu/pick/27951,...,-149.5225,55.8575,12800.0,2.4,ml,33,-152.437500,57.640100,79.0,m/s
403727,S,269.35,2.328,1.813,0.131,2018-10-31T20:04:48.557000Z,XO,WD55,HH1,quakeml:earthquake.alaska.edu/pick/27952,...,-149.5225,55.8575,12800.0,2.4,ml,33,-153.662817,55.761625,-1283.5,m/s
403728,S,320.25,2.552,-2.620,0.121,2018-10-31T20:04:49.984000Z,II,KDAK,BH1,quakeml:earthquake.alaska.edu/pick/27953,...,-149.5225,55.8575,12800.0,2.4,ml,33,0.000000,0.000000,0.0,
403729,S,320.77,2.637,-1.081,0.118,2018-10-31T20:04:53.576000Z,XO,KD02,HHN,quakeml:earthquake.alaska.edu/pick/27954,...,-149.5225,55.8575,12800.0,2.4,ml,33,-152.657500,57.861800,57.0,m/s


In [156]:
alll.to_parquet('alaska_fullcatalog.parquet',version='2.6')